In [1]:
%reset -f

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import tensorflow as tf


In [3]:
df = pd.read_pickle('test_pairs.pkl')

mask = []

#removing empty entries from uncommon2

for index,words in enumerate(df.Uncommon_Words_Query_2):
    
    if len(words) == 0:
        mask.append(index)

df = df.drop(mask)
df.head()

,Query_1,Query_2,Common_Words,Uncommon_Words_Query_1,Uncommon_Words_Query_2,Intent_1,Intent_2
0,who will a approve my <t>,who to connect with for <t> balance upgradation,[who],[approve],[balance],who will approve my leave,who will approve my leave
1,who will a approve my <t>,who is the poc for <t> related issues,[who],[approve],[issue],who will approve my leave,who will approve my leave
2,who will a approve my <t>,i have to delete my time booking which is appr...,[approve],[who],"[time, which, want, apply]",who will approve my leave,who will approve my leave
3,who will a approve my <t>,is there anyone who should be contacted if the...,[who],[approve],"[anyone, contact, mismatch]",who will approve my leave,who will approve my leave
4,who will a approve my <t>,do you know who is the <t> <t> to update <t> b...,[who],[approve],"[update, balance]",who will approve my leave,who will approve my leave


In [4]:
# df = pd.read_pickle('query_pairs_fixed.pkl')
# df.head()

In [5]:
#loading data into lists

n = df.shape[0]

#n = 1000 #partial data for checking implementation

intent_1 = list(df.Intent_1.iloc[:n])    
intent_2 = list(df.Intent_2.iloc[:n])

uncommon1 = list(df.Uncommon_Words_Query_1.iloc[:n])
uncommon2 = list(df.Uncommon_Words_Query_2.iloc[:n])

common = list(df.Common_Words.iloc[:n])

In [6]:
#Create a dictionary of intent types to initialize embeddings

unique_intents = list(set(intent_1 + intent_2))
print('Total unique intents = {}'.format(len(unique_intents)))
intent_index = list(np.arange(0, len(unique_intents)))

intent_dict = dict(zip(unique_intents, intent_index))

Total unique intents = 5


In [7]:
#create dictionary of vocabulary (other permanent dictionary stored in the Dictionaries.py file)

def get_vocab_dict(lol):
    
    #lol = list of lists (of strings)

    vocab = []
    for words in lol:
        
        for word in words:
            
            vocab.append(word)
            
    vocab = list(set(vocab))
    values = list(np.arange(0, len(vocab)))
    
    vocab_dict = dict(zip(vocab, values))
            
    return vocab_dict

In [8]:
#create vocabulary dictionary of all words 

vocab_dict = get_vocab_dict(uncommon1 + uncommon2)
len(vocab_dict)

116

In [9]:
#Convert words in uncommon1 and uncommon2 to their respective indices in vocab_dict

unc1_index = []
unc2_index = []

for words  in uncommon1:

    indices1 = []
    for word in words:
        
        dict_value = vocab_dict[word]
        indices1.append(dict_value)
        
    unc1_index.append(indices1)

    
for words in uncommon2:
    
    indices2 = []
    for word in words:
        
        dict_value = vocab_dict[word]
        indices2.append(dict_value)
        
    unc2_index.append(indices2)

In [10]:
#Convert named intents to their corresponding value in the dictionary

intent1_index = []
intent2_index = []

for intent in intent_1:
     
    l = []
    intent_value = intent_dict[intent]
    l.append(intent_value)
    intent1_index.append(l)
    
for intent in intent_2:
    
    l = []
    intent_value = intent_dict[intent]
    l.append(intent_value)
    intent2_index.append(l)
    
    #Get intent indices for negative samples
    
intent1_index = np.asarray(intent1_index)
intent2_index = np.asarray(intent2_index)

negative_index = []

for i in range(len(intent2_index)):
    
    l = []
    negative_intent = np.random.randint(0, len(unique_intents) - 1)
    if negative_intent >= intent2_index[i]:
        
        negative_intent += 1
        
    l.append(negative_intent)
    negative_index.append(l)

negative_index = np.asarray(negative_index)

In [11]:
#initialize embeddings for intents and words (the current initialization used is from OpenKE's KE model)

embed_dim = 8

intent_embeddings = tf.get_variable(name = 'intent_embeddings', shape = [len(unique_intents), embed_dim], initializer = 
                                    tf.contrib.layers.xavier_initializer(uniform = False))

word_embeddings = tf.get_variable(name = 'word_embeddings', shape = [len(vocab_dict), embed_dim], initializer = 
                                    tf.contrib.layers.xavier_initializer(uniform = False))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
#--------------------------------------------------------------------------------
#SINGLE INSTANCE BATCH
#--------------------------------------------------------------------------------
u1_batch = tf.placeholder(tf.int32, shape=[None, ])
u2_batch = tf.placeholder(tf.int32, shape=[None, ])

i1_batch = tf.placeholder(tf.int32, shape=[None, ])
i2_batch = tf.placeholder(tf.int32, shape=[None, ])
n_batch  = tf.placeholder(tf.int32, shape=[None, ])

#generating training examples using embedding_lookup
i1 = tf.nn.l2_normalize(tf.nn.embedding_lookup(intent_embeddings, i1_batch), axis=1)
i2 = tf.nn.l2_normalize(tf.nn.embedding_lookup(intent_embeddings, i2_batch), axis=1)
i3 = tf.nn.l2_normalize(tf.nn.embedding_lookup(intent_embeddings, n_batch), axis=1)
 
unc1_vector = tf.reduce_sum(tf.nn.l2_normalize(tf.nn.embedding_lookup(word_embeddings, u1_batch),axis=1), 0)
unc2_vector = tf.reduce_sum(tf.nn.l2_normalize(tf.nn.embedding_lookup(word_embeddings, u1_batch),axis=1), 0)

epsilon = 1e-15

#loss = tf.nn.l2_loss(i1 - i2)
#loss = tf.norm((i1 - i2) - (unc1_vector - unc2_vector)) - tf.norm((i1 - i3) - (unc1_vector - unc2_vector))
loss = tf.norm((i1 - i2 + epsilon) - (unc1_vector - unc2_vector)) - tf.norm((i1 - i3 + epsilon) - (unc1_vector - unc2_vector)) 
#loss = tf.norm((i1 - unc1_vector)) + tf.norm((i2 - unc2_vector))


#tf.trainable_variables()
optimizer  = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(loss)


Instructions for updating:
Use tf.cast instead.


In [13]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    
    sess.run(init)
    
    init_loss = 0
    final_loss = 0
    
#    for epochs in (range(7)):
    

    for i in tqdm(range(100)):


        feed_dict = {i1_batch : intent1_index[i], i2_batch : intent2_index[i], n_batch : negative_index[i],
                     u1_batch : unc1_index[i], u2_batch : unc2_index[i]}


        init_loss += sess.run(loss, feed_dict)
        sess.run(train_op, feed_dict)
        final_loss += sess.run(loss, feed_dict)


    intent_vectors = sess.run(intent_embeddings)
    word_vectors = sess.run(word_embeddings)


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 79.60it/s]


In [14]:
init_loss, final_loss

(-124.95627278089523, -125.80549603700638)

In [15]:
#save obtained vectors as numpy arrays to be used in future

# np.save('word_vectors', word_vectors)
# np.save('intent_vectors', intent_vectors)

In [16]:
#convert obtained vectors to TSV files for visualization using embedding projector (http://projector.tensorflow.org/)

np.savetxt('w_vec.txt', word_vectors , delimiter='\t')
np.savetxt('i_vec.txt', intent_vectors , delimiter='\t')

In [17]:
#saving word and intent keys for obtained vectors to be used as metadata for the visualizer

import csv   

wkeys = []

for key in vocab_dict.keys():
    wkeys.append(key)
    
      
with open('w_keys.txt', 'w') as f_output:
    tsv_output = csv.writer(f_output, delimiter='\n')
    tsv_output.writerow(wkeys)
    


In [18]:
ikeys = []

for key in intent_dict.keys():
    ikeys.append(key)
    

with open('i_keys.txt', 'w') as f_output:
    tsv_output = csv.writer(f_output, delimiter='\n')
    tsv_output.writerow(ikeys)

In [19]:
#appending both intent and word vectors for combined visualization 

main = np.concatenate((word_vectors, intent_vectors), axis=0)
np.savetxt('main.txt', main , delimiter='\t')

#metadata for combined visualization

main_keys = []

for keys in vocab_dict.keys():  
    main_keys.append(keys)
    
for keys in intent_dict.keys(): 
    main_keys.append(keys)
    
    
with open('main_keys.txt', 'w') as f_output:
    tsv_output = csv.writer(f_output, delimiter='\n')
    tsv_output.writerow(main_keys)

In [ ]:
#------------------------------FOR EVALUTATION OF VECTORS-----------------------------------------------

In [ ]:
def w2v(word):
    
    word_index = vocab_dict[word]
    word_vector = word_vectors[word_index]
    
    return word_vector

def intent2vec(intent):
    
    intent_index = intent_dict[intent]
    intent_vector = intent_vectors[intent_index]
    
    return intent_vector

np.set_printoptions(suppress=True)

In [ ]:
vocab_dict['how']

In [ ]:
approval = w2v('approval')     #w2v function finds the embedding of the word in the trained embeddings
left = w2v('left')
remain = w2v('remain')
step = w2v('step')
apply = w2v('apply')
take = w2v('take')
procedure = w2v('procedure')
how = w2v('how')
many = w2v('many')
encash = w2v('encash')
balance = w2v('balance')
encashment = w2v('encashment')
year = w2v('year')
beginning = w2v('beginning')
provide = w2v('provide')
new = w2v('new')
employee = w2v('employee')
month = w2v('month')
every = w2v('every')
adopt = w2v('adopt')
birth = w2v('birth')
second = w2v('second')
third = w2v('third')
application = w2v('application')
process = w2v('process')



many_kids = intent2vec('will i get maternity leave for more than two children')    #intent2vec finds embedding of intent in trained embeds
intent_vec2 = intent2vec('how to apply leave')


In [ ]:
remain , left

In [ ]:
a = [1,1]
b = [2,2]
cosine(a,b)

In [ ]:
adopt + birth + second + third , many_kids

In [ ]:
many + provide + every + month

In [ ]:
beginning + new + year + many + provide + employee

In [ ]:

(approval + left) , intent_vectors[12]

In [ ]:
from scipy.spatial.distance import cosine

dists = []

e1 = intent_vectors[18]

for index, intent in enumerate(intent_vectors):
    
    if index == 18:
        
        continue
        
    dists.append(cosine(e1, intent))


In [ ]:
min(dists)

In [ ]:
for index, val in enumerate(dists):
    
    if val == min(dists):

        print(index)

In [ ]:
intent_dict

In [ ]:
#np.save('Intent_Embeddings', intent_vectors, allow_pickle=True)
#np.save('Word_Embeddings', word_vectors, allow_pickle=True)

In [ ]:
from nltk import FreqDist

def bow(text):
    
    all_words = []
    
    for sentence in text:
        
        for word in sentence:
            
            all_words.append(word.lower())
            
    all_words = FreqDist(all_words)

    return all_words

In [ ]:
words8 = np.load('Word_Embeddings_8.npy', allow_pickle=True)
intents8 = np.load('Intent_Embeddings_8.npy', allow_pickle=True)

In [ ]:
intents8

In [ ]:
words30 = np.load('Word_Embeddings.npy', allow_pickle=True)
intents30 = np.load('Intent_Embeddings.npy', allow_pickle=True)

In [ ]:
qualify = w2v('qualify')
eligible = w2v('eligible')

In [ ]:
np.linalg.norm(qualify - apply, 2)

In [ ]:
#---------------------------POSSIBLE APPROACHED TO CREATE BATCHES (Work in Progress)--------------------------

In [ ]:
'''
#using padding to make all arrays of same lenght and then retrieving the required indices 
#(possible method to create batches for un1, unc2), needs further investigation

a = tf.convert_to_tensor(np.array([0,1,2,3,0]))
b = tf.placeholder(tf.int32, shape=[None, 5])

data_mask = tf.cast(a, tf.bool)
init = tf.global_variables_initializer()

sess = tf.Session()

sess.run(init)
print(sess.run(b, {b:sess.run(a)}))
for i in b:
    
    print(sess.run(i))
print(tf.shape(data_mask))
print(sess.run(data_mask))
print(sess.run(vals))

'''

In [ ]:
#def get_batch(batch_size):

In [ ]:
'''
start = tf.placeholder(tf.int32, shape=(1,))
end = tf.placeholder(tf.int32, shape=(1,))

current_pos = 0
batch_size = 5

def get_batch(current_pos, batch_size):
    
    start_pos = current_pos
    end_pos = current_pos + batch_size
    
    last = end_pos

    return start_pos, end_pos, last

asd = tf.global_variables_initializer()


with tf.Session() as s:
    
    s.run(asd)
    start, end, last = get_batch(current_pos, batch_size)
    intent1_batch = intent1_index[:end]
    print(intent1_batch)
#i1 = tf.nn.embedding_lookup(intent_embeddings, intent1_index)
'''


In [ ]:
'''

#finding lenght of each training instance


unc1_len = []

for item in unc1_index:
    unc1_len.append(len(item))


#---------------------------------------

unc2_len = []

for item in unc2_index:
    unc2_len.append(len(item))
    
unc2_len[:3], unc1_len[:3]

'''

In [ ]:
'''
padded_size = 9

all_zero = np.zeros((padded_size)).astype(int)
unc1_padded = []
for item in unc1_index:
    
    copy = all_zero.copy()
    copy[:len(item)] = item
    unc1_padded.append(copy)
    
unc1_padded = np.asarray(unc1_padded)
unc1_padded[:6]

'''

In [ ]:
'''

padded_size = 9

all_zero = np.zeros((padded_size)).astype(int)
unc2_padded = []
for item in unc2_index:
    
    copy = all_zero.copy()
    copy[:len(item)] = item
    unc2_padded.append(copy)
    
unc2_padded = np.asarray(unc2_padded)
unc2_padded[:6]

'''

In [ ]:
'''
def get_batch(start, batch_size):
    
    initial = start
    final = start + batch_size
    
    current_pos = final
    
    return initial, final, current_pos


batch_size = 64
current_pos = 0

num_batches = n // batch_size
'''

In [ ]:
'''

u1 = tf.placeholder(tf.int32, shape=[None, 9])
u2 = tf.placeholder(tf.int32, shape=[None, 9])

i1_batch = tf.placeholder(tf.int32, shape=[None, ])
i2_batch = tf.placeholder(tf.int32, shape=[None, ])
n_batch = tf.placeholder(tf.int32, shape=[None, ])

i1 = tf.nn.embedding_lookup(intent_embeddings, i1_batch)
i2 = tf.nn.embedding_lookup(intent_embeddings, i2_batch)
negative = tf.nn.embedding_lookup(intent_embeddings, n_batch)



loss = tf.reduce_sum(tf.norm((i1-i2) - (unc1 - unc2)) - tf.norm((i1-negative) - (unc1 - unc2)))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)


init = tf.global_variables_initializer()


with tf.Session() as s:
    
    s.run(init)
    
    for j in range(5):
        
        start, end, current_pos = get_batch(current_pos, batch_size)
        print(start, end)
        
#-----------------------------------------------------------------------------------

        mask1 = s.run(u1, {u1: unc1_padded[start:end]})
        unc1 = []
        for index, val in enumerate(unc1_len[start:end]):
            
            e_id = mask1[index][:val]
            emb = tf.nn.embedding_lookup(word_embeddings, e_id)
            one_sum = tf.reduce_sum(emb, 0)
            unc1.append(one_sum)
            
        unc1 = tf.convert_to_tensor(unc1)
        
#-----------------------------------------------------------------------------------
        
        mask2 = s.run(u2, {u2: unc2_padded[start:end]})

        unc2 = []
        for index, val in enumerate(unc2_len[start:end]):
            
            e_id = mask2[index][:val]
            emb = tf.nn.embedding_lookup(word_embeddings, e_id)
            one_sum = tf.reduce_sum(emb, 0)
            unc2.append(one_sum)
            
        unc2 = tf.convert_to_tensor(unc2)
    
#------------------------------------------------------------------------------------
        feed_dict={i1_batch : intent1_index[start:end], i2_batch : intent2_index[start:end],
                   n_batch : negative_index[start:end]}
    
#---------------------------------------------------------------------------------------
            
        
        
    
        s.run(optimizer, feed_dict)
        
    intent_vectors = s.run(intent_embeddings)
    word_vectors = s.run(word_embeddings)
        
'''

In [ ]:




#---------------------ACTUAL CODE ENDS HERE, ONY TESTS BEYOND THIS POINT----------------------------------------------





In [ ]:
'''

#this works and can be used to create batches for i1, i2, i3 but not for unc1, unc2

i1_index = tf.placeholder(tf.int32, shape=[None])

i1 = tf.nn.embedding_lookup(intent_embeddings, i1_index)

loss = tf.norm(i1)

init = tf.global_variables_initializer()

optimizer  = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(loss)

def get_batch(start, batch_size):
    
    initial = start
    final = start + batch_size
    
    current_pos = final
    
    return initial, final, current_pos

batch_size = 5
current_pos = 0

for i in range(2):
    
    start, end, current_pos = get_batch(current_pos, batch_size)
    print(start, end)
    
    

with tf.Session() as sess:
    
    sess.run(init)
    batch_size = 5
    current_pos = 0

    print(sess.run(i1, feed_dict={i1_index: intent2_index[:10]}))
    print('---------------------')
    
    for j in range(2):
        
        start, end, current_pos = get_batch(current_pos, batch_size)
        print(sess.run(i1, feed_dict={i1_index: intent2_index[start:end]}))
        print(sess.run(loss, feed_dict={i1_index: intent2_index[start:end]}))
        
        for i in range(100):
            sess.run(train_op, feed_dict={i1_index: intent2_index[start:end]})
            
        print(sess.run(loss, feed_dict={i1_index: intent2_index[start:end]}))
        print(sess.run(i1, feed_dict={i1_index: intent2_index[start:end]}))
        
    print('-------------------------')  
    print(sess.run(i1, feed_dict={i1_index: intent2_index[:10]}))
    
'''

In [ ]:
'''

#trying methods to create batches for unc1, unc2

i1_index = tf.placeholder(tf.int32, shape=[None])
i1 = tf.nn.embedding_lookup(intent_embeddings, i1_index)

u2_index = tf.placeholder(tf.int32, shape=(5,))
u2 = tf.unstack(tf.reshape(u2_index, [-1])) #tf cannot iterate over tensor object so using stack to get around that for now



init = tf.global_variables_initializer()

with tf.Session() as sess:
    
    sess.run(init)
    print('initial embedding ', sess.run(word_embeddings[3]))
    
    
    batch_u2 = unc2_len[:batch_size]
    unc2_vector = []
    count = 0
    for size in u2:
        
        l = int(sess.run(size, {u2_index: batch_u2}))   #choosing some instances to train
        end = count + l  

        word_index = unc2_concat[:l]   #somehow 'l' works but replacing 'l' with 'end' does not work, need to look at error
        w_vec = tf.nn.embedding_lookup(word_embeddings, word_index)
        one_sum = tf.reduce_sum(w_vec, 0)
        unc2_vector.append(one_sum)
        count += size
        
    print(sess.run(unc2_vector))
        
    loss = tf.norm(i1 - unc2_vector)
    optimizer  = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(loss)
    
    print(sess.run(loss))
    for i in range(1000):
        
        sess.run(train_op, {i1_index : intent1_index[:5]})
    
    print(sess.run(loss))
    print('trained embedding ',sess.run(word_embeddings[0]))
        
    

    
    
#this sort of works but need to figure out how to implement in sync with rest of the code

'''

In [ ]:
'''
#generating training examples using embedding_lookup


i1 = tf.nn.embedding_lookup(intent_embeddings, intent1_index)
i2 = tf.nn.embedding_lookup(intent_embeddings, intent2_index)
i3 = tf.nn.embedding_lookup(intent_embeddings, negative_index)
   
#adding embeddings of individual words for a training example to form Σuc1, Σuc2
 
unc1_vector = []
for mask in unc1_index:
       
    w_vec = (tf.nn.embedding_lookup(word_embeddings, mask))
    one1_sum = tf.reduce_sum(w_vec, 0)
    unc1_vector.append(one1_sum)
       
unc1_vector = tf.convert_to_tensor(unc1_vector)

        
unc2_vector = []
for mask in unc2_index:
        
    u_vec = (tf.nn.embedding_lookup(word_embeddings, mask))
    one2_sum = tf.reduce_sum(u_vec, 0)
    unc2_vector.append(one2_sum)
        
unc2_vector = tf.convert_to_tensor(unc2_vector)


#loss = tf.reduce_sum(abs((a-b) - (c-d)) - abs((a-e) - (c-d)))
loss = tf.reduce_sum(tf.norm((i1-i2) - (unc1_vector - unc2_vector)) - tf.norm((i1-i3) - (unc1_vector - unc2_vector)))

#loss = tf.nn.l2_loss((i1 - unc1_vector))



#tf.trainable_variables()
optimizer  = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(loss)

'''

In [ ]:
#--------------------------------------------------------------------------------
#SINGLE INSTANCE BATCH
#--------------------------------------------------------------------------------
u1_batch = tf.placeholder(tf.int32, shape=[None,])
u2_batch = tf.placeholder(tf.int32, shape=[None,])

i1_batch = tf.placeholder(tf.int32, shape=[None, ])
i2_batch = tf.placeholder(tf.int32, shape=[None, ])
n_batch = tf.placeholder(tf.int32, shape=[None, ])

#generating training examples using embedding_lookup



i1 = tf.nn.embedding_lookup(intent_embeddings, i1_batch)
i2 = tf.nn.embedding_lookup(intent_embeddings, i2_batch)
i3 = tf.nn.embedding_lookup(intent_embeddings, n_batch)
   
#adding embeddings of individual words for a training example to form Σuc1, Σuc2
 
unc1_vector = tf.reduce_sum((tf.nn.embedding_lookup(word_embeddings, u1_batch)), 0)

       
unc2_vector = tf.reduce_sum((tf.nn.embedding_lookup(word_embeddings, u2_batch)), 0)



#loss = tf.reduce_sum(abs((a-b) - (c-d)) - abs((a-e) - (c-d)))
loss = tf.reduce_sum(tf.norm((i1-i2) - (unc1_vector - unc2_vector)) - tf.norm((i1-i3) - (unc1_vector - unc2_vector)))

#loss = tf.nn.l2_loss((i1 - unc1_vector))



#tf.trainable_variables()
optimizer  = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(loss)

In [ ]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    
    sess.run(init)
    print(sess.run(word_embeddings[0:5]))

    
#    print(sess.run(loss, {i1_batch : intent1_index, i2_batch : intent2_index, n_batch : negative_index,
#                u1_batch : unc1_index, u2_batch : unc2_index}))

    init_loss = 0
    final_loss = 0
    
    for i in range(1000):
        
        feed_dict = {i1_batch : intent1_index[i], i2_batch : intent2_index[i], n_batch : negative_index[i],
                     u1_batch : unc1_index[i], u2_batch : unc2_index[i]}
        
        init_loss += sess.run(loss, feed_dict)
        sess.run(train_op, feed_dict)
        final_loss += sess.run(loss, feed_dict)
        
        if i%2000 == 0:
            
            print(i)
        
    print(sess.run(word_embeddings[:5]))


    intent_vectors = sess.run(intent_embeddings)
    word_vectors = sess.run(word_embeddings)


In [ ]:
def word2vec(word):
    
    word_index = vocab_dict[word]
    word_vector = word_vectors[word_index]
    
    return word_vector

def intent2vec(intent):
    
    intent_index = intent_dict[intent]
    intent_vector = intent_vectors[intent_index]
    
    return intent_vector

In [ ]:
np.dot((step + apply) , (procedure + take))     #step + apply ~= procedure + take

In [ ]:
step + apply - take    #(q1,q2) = (step to apply leave, procedure to take leave) --- (step == procedure, apply == take)
                        # which means step + apply - take == procedure

In [ ]:
procedure     #as can be seen here (used dataset is small and training iterations are more so values are really close)

In [ ]:
     #intent1 - intent2 == unc1 - unc2

In [ ]:
#    print(sess.run(i1[0]))
#    print(sess.run(intent_embeddings[intent1_index[0]]))
#    print(sess.run(word_embeddings[unc1_index[0][0]]))
    
#    print('embedding of intent 1 - ', sess.run(intent_embeddings[intent1_index[0]]))
#    print('embedding of unc1 words - ', sess.run(word_embeddings[unc1_index[0][0]]))
#    print(sess.run(word_embeddings[unc1_index[0][1]]))
#    print('first five word embeddings - ')
#    old = sess.run(word_embeddings)
#    print(old)
#    print('-------------')
#    print('initially i1 - unc1 -', sess.run(i1 - unc1_vector)[0])
#    for batch_num in range(0, n, batch_size):
#---------------------------------------------

#        print(sess.run((i1, i2, i3, unc1_vector, unc2_vector), feed_dict ))
#        sess.run(train_op, feed_dict)  
        
#    print(sess.run(loss, {i1_batch : intent1_index, i2_batch : intent2_index, n_batch : negative_index,
#                u1_batch : unc1_index, u2_batch : unc2_index}))
    
#    print('\n')
#    print('embedding of intent 1 - ', sess.run(intent_embeddings[intent1_index[0]]))
#    print('embedding of unc1 words - ', sess.run(word_embeddings[unc1_index[0][0]]))
#    print(sess.run(word_embeddings[unc1_index[0][1]]))
#    print('updated word embeddings')
#    new = sess.run(word_embeddings)
#    print(old - new)
    
#    print('---------------')
#    print('finally i1 - unc1 ', sess.run(i1 - unc1_vector)[0])

#    print(sess.run(i1 - unc1_vector))

#    for epoch in range(epochs)